# Proyecto 2

## Diego Franco - 20240

In [1]:
import pandas as pd

#Lee el archivo CSV
data = pd.read_csv('BalanceData.csv')

In [2]:
from sklearn.preprocessing import LabelEncoder

# Eliminar la columna 'trans_date_trans_time'
data.drop(columns=['trans_date_trans_time', 'transaction_time'], inplace=True)

# Codificar variables categóricas con label encoding
label_encoder = LabelEncoder()
categorical_cols = ['merchant', 'category', 'first', 'last', 'gender', 'street', 'city', 'state', 'job']
for col in categorical_cols:
    data[col + '_encoded'] = label_encoder.fit_transform(data[col])

# Eliminar columnas originales no numéricas y otras columnas irrelevantes
data.drop(columns=['merchant', 'category', 'first', 'last', 'gender', 'street', 'city', 'state', 'job', 'dob', 'trans_num'], inplace=True)



In [3]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer 

# Dividir el dataset en conjunto de entrenamiento, conjunto de validación (dev) y conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(data.drop('is_fraud', axis=1), data['is_fraud'], test_size=0.2, random_state=42)
X_train, X_dev, y_train, y_dev = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

# Imputar valores faltantes con la estrategia de relleno con el valor medio
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_dev_imputed = imputer.transform(X_dev)


## Creacion de los modelos

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score
from sklearn.impute import SimpleImputer 
import joblib

from sklearn.neural_network import MLPClassifier
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

def evaluate_model(model, X, y):
    # Predecir probabilidades
    y_pred_proba = model.predict_proba(X)[:, 1] if hasattr(model, 'predict_proba') else model.decision_function(X)
    
    # Calcular métricas
    roc_auc = roc_auc_score(y, y_pred_proba)
    precision = precision_score(y, model.predict(X))
    recall = recall_score(y, model.predict(X))
    f1 = f1_score(y, model.predict(X))
    
    return roc_auc, precision, recall, f1

# Inicializar los modelos
ann_model = MLPClassifier()
lgb_model = lgb.LGBMClassifier()
xgb_model = xgb.XGBClassifier()
rf_model = RandomForestClassifier()
svm_model = SVC(probability=True)  # Habilitar la predicción de probabilidades para SVM

# Diccionario de modelos
models = {
    #"LightGBM": lgb_model,
    #"XGBoost": xgb_model,
    #"Random Forest": rf_model,
    "SVM": svm_model
}

# Imputar valores faltantes con la estrategia de relleno con el valor medio
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_dev_imputed = imputer.transform(X_dev)

# Diccionario para almacenar métricas antes y después del entrenamiento incremental
metrics_before_incremental = {}
metrics_after_incremental = {}

# Iterar sobre cada modelo
for model_name, model in models.items():

    print(f"Entrenando modelo {model_name}...")
    # Entrenamiento inicial
    model.fit(X_train_imputed, y_train)  # Utilizar X_train_imputed

    # Guardar el modelo antes del entrenamiento incremental
    initial_model_filename = f"{model_name}_initial_model.pkl"
    joblib.dump(model, initial_model_filename)
    print(f"Modelo {model_name} guardado exitosamente.")
    
    # Evaluación inicial
    roc_auc_before, precision_before, recall_before, f1_before = evaluate_model(model, X_dev_imputed, y_dev)  # Utilizar X_dev_imputed
    metrics_before_incremental[model_name] = (roc_auc_before, precision_before, recall_before, f1_before)
    
    # Entrenamiento incremental con nuevos datos nunca antes utilizados por el modelo
    X_new_batch, _, y_new_batch, _ = train_test_split(data.drop('is_fraud', axis=1), data['is_fraud'], test_size=0.1, random_state=42)
    X_new_batch_imputed = imputer.transform(X_new_batch)
    model.partial_fit(X_new_batch_imputed, y_new_batch)

    # Guardar el modelo después del entrenamiento incremental
    incremental_model_filename = f"{model_name}_incremental_model.pkl"
    joblib.dump(model, incremental_model_filename)
    print(f"Modelo {model_name} guardado exitosamente.")
    
    # Evaluación después del entrenamiento incremental
    roc_auc_after, precision_after, recall_after, f1_after = evaluate_model(model, X_dev_imputed, y_dev)  # Utilizar X_dev_imputed
    metrics_after_incremental[model_name] = (roc_auc_after, precision_after, recall_after, f1_after)

    print(f"Modelo {model_name} entrenado y evaluado exitosamente.")
    print()
    print("Antes del entrenamiento incremental:")
    print(f"ROC-AUC: {roc_auc_before}, Precisión: {precision_before}, Recall: {recall_before}, F1-score: {f1_before}")
    print("Después del entrenamiento incremental:")
    print(f"ROC-AUC: {roc_auc_after}, Precisión: {precision_after}, Recall: {recall_after}, F1-score: {f1_after}")

# Imprimir las métricas antes y después del entrenamiento incremental para cada modelo
for model_name, (roc_auc_before, precision_before, recall_before, f1_before), (roc_auc_after, precision_after, recall_after, f1_after) in zip(metrics_before_incremental.keys(), metrics_before_incremental.values(), metrics_after_incremental.values()):
    print(f"Modelo: {model_name}")
    print("Antes del entrenamiento incremental:")
    print(f"ROC-AUC: {roc_auc_before}, Precisión: {precision_before}, Recall: {recall_before}, F1-score: {f1_before}")
    print("Después del entrenamiento incremental:")
    print(f"ROC-AUC: {roc_auc_after}, Precisión: {precision_after}, Recall: {recall_after}, F1-score: {f1_after}")
    print()


Entrenando modelo SVM...


In [ ]:
# Convertir los resultados antes del entrenamiento incremental a un DataFrame
results_before_df = pd.DataFrame({
    'Modelo': list(metrics_before_incremental.keys()),
    'ROC-AUC': [metric[0] for metric in metrics_before_incremental.values()],
    'Precisión': [metric[1] for metric in metrics_before_incremental.values()],
    'Recall': [metric[2] for metric in metrics_before_incremental.values()],
    'F1-score': [metric[3] for metric in metrics_before_incremental.values()]
})

# Guardar el DataFrame de resultados antes del entrenamiento incremental en un archivo CSV
results_before_df.to_csv('resultados_before_entrenamiento_incremental.csv', index=False)

# Convertir los resultados después del entrenamiento incremental a un DataFrame
results_after_df = pd.DataFrame({
    'Modelo': list(metrics_after_incremental.keys()),
    'ROC-AUC': [metric[0] for metric in metrics_after_incremental.values()],
    'Precisión': [metric[1] for metric in metrics_after_incremental.values()],
    'Recall': [metric[2] for metric in metrics_after_incremental.values()],
    'F1-score': [metric[3] for metric in metrics_after_incremental.values()]
})

# Guardar el DataFrame de resultados después del entrenamiento incremental en un archivo CSV
results_after_df.to_csv('resultados_after_entrenamiento_incremental.csv', index=False)